In [ ]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 412.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=a4a0950023a1e214c9fdf8438c28bdeab0f805e7b6a7d4717dca9aeace1d81f8
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=b59a8f4379fd819b9fab84f763eff8a07936d5509d95cceae989f2749e3f3ba0
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import time
import requests
from torchvision.datasets import ImageFolder
from torchvision import models
from PIL import Image
from shutil import copyfile
import os

os.chdir('/content/drive/My Drive')
from datasets.cityscapes import CityscapesDataset
from datasets.gta5 import GTA5

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((720,1280), interpolation=Image.Resampling.NEAREST),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_lbl_transform = transforms.Compose([
    transforms.Resize((720,1280), interpolation=Image.Resampling.NEAREST),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((512,1024), interpolation=Image.Resampling.NEAREST),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_lbl_transform = transforms.Compose([
    transforms.Resize((512,1024), interpolation=Image.Resampling.NEAREST),
])

In [ ]:
train_dataset = GTA5('/content/drive/MyDrive/datasets/GTA5', transform=[train_transform,train_lbl_transform]) # datasets\\GTA5
val_dataset = CityscapesDataset('/content/drive/MyDrive/datasets/Cityscapes/Cityscapes/Cityspaces', transform=[val_transform,val_lbl_transform]) # datasets\\Cityscapes\\Cityspaces

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers = 8) #16
val_dataloader = DataLoader(val_dataset, split='val', batch_size=4, shuffle=True, num_workers = 8) #16

In [ ]:
for images, labels in train_dataloader:
    # Access the first image and label
    image, label = images[0], labels[0]
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)
    print("Label:", label)

    break

Image shape: torch.Size([3, 720, 1280])
Label shape: torch.Size([720, 1280])
Label: tensor([[ 10,  10,  10,  ...,   8,   8,   8],
        [ 10,  10,  10,  ...,   8,   8,   8],
        [ 10,  10,  10,  ...,   8,   8,   8],
        ...,
        [  0,   0,   0,  ..., 255, 255, 255],
        [  0,   0,   0,  ..., 255, 255, 255],
        [  0,   0,   0,  ..., 255, 255, 255]])


In [ ]:
for images, labels in val_dataloader:
    # Access the first image and label
    image, label = images[0], labels[0]
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)
    print("Label:", label)

    break

Image shape: torch.Size([3, 512, 1024])
Label shape: torch.Size([512, 1024])
Label: tensor([[255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        ...,
        [255, 255, 255,  ...,   0, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255]])


In [ ]:
# Load pre-trained model

from models.bisenet.build_bisenet import BiSeNet
bisenet = BiSeNet(num_classes = 19, context_path='resnet18').to(device)

In [ ]:
bisenet

BiSeNet(
  (saptial_path): Spatial_path(
    (convblock1): ConvBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (convblock2): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (convblock3): ConvBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (context_path): resnet18(
    (features): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [ ]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
# Get the length of class_names (one output unit for each class)
output_shape = 19 # 0 to 18

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
initial_lr = 2.5e-2/4
optimizer = torch.optim.SGD(bisenet.parameters(), lr=initial_lr, momentum=0.9, weight_decay=1e-4)

In [ ]:
from models.utils import fast_hist
from models.utils import per_class_iou
# from utils import fast_hist
# from utils import per_class_iou

def total_hist(outputs, labels, num_classes):
    hist = 0
    for i in range(len(outputs)):
        output, label = outputs[i].cpu().detach().numpy().reshape(-1,), labels[i].cpu().detach().numpy().reshape(-1,)
        hist += fast_hist(label, output, num_classes)
    return hist

In [ ]:
def train(model, optimizer, dataloader, loss_fn, alpha = 1):

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    hist = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        # targets = torch.argmax(targets, dim=1) # remember that this is necessary because the loss function expects differents shapes (batch_size, n_classes, height, width) for pred and (batch_size, height, width) for target

        outputs, cx1_sup, cx2_sup = model(inputs)

        # main loss
        loss = loss_fn(outputs, targets)

        # # supervision losses
        # sup1_loss = loss_fn(cx1_sup, targets)
        # sup2_loss = loss_fn(cx2_sup, targets)

        # # combined loss
        # loss = main_loss + alpha * (sup1_loss + sup2_loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        hist += total_hist(predicted, targets, 19)

    train_loss = running_loss / len(dataloader)
    miou_per_class = per_class_iou(hist)
    miou = np.mean(miou_per_class)

    return train_loss, miou, miou_per_class


def test(model, dataloader, loss_fn):

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    hist = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            # targets = torch.argmax(targets, dim=1)

            outputs = model(inputs)

            loss = loss_fn(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)

            hist += total_hist(predicted, targets, 19)

    test_loss = test_loss / len(dataloader)

    miou_per_class = per_class_iou(hist)
    miou = np.mean(miou_per_class)

    return test_loss, miou, miou_per_class

In [ ]:
from models.utils import poly_lr_scheduler
# from utils import poly_lr_scheduler

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
epochs = 5 #50

miou_train_list = []
miou_test_list = []
miou_per_class_train_list = []
miou_per_class_test_list = []

for ep in range(epochs):

    train_loss, train_miou, train_miou_per_class = train(bisenet, optimizer, train_dataloader, loss_fn)
    test_loss, test_miou, test_miou_per_class = test(bisenet, val_dataloader, loss_fn)

    ## Inserting learning rate decay
    curr_lr = poly_lr_scheduler(optimizer = optimizer, init_lr = initial_lr, iter = ep, lr_decay_iter=1, max_iter=epochs, power=0.9)

    miou_train_list.append(train_miou)
    miou_test_list.append(test_miou)
    miou_per_class_train_list.append(train_miou_per_class)
    miou_per_class_test_list.append(test_miou_per_class)

    print(f"Current learning rate: {curr_lr}")
    print(f"Test loss: {test_loss}")
    print(f"Train mIoU: {train_miou}")
    print(f"Test mIoU: {test_miou}")
    print("\n")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

Current learning rate: 0.00625
Test loss: 1.946525250075731
Train mIoU: 0.09953300963256177
Test mIoU: 0.05214746235555472


Current learning rate: 0.005112825912817865
Test loss: 1.679348918014507
Train mIoU: 0.10372235722472083
Test mIoU: 0.056841860290313864


Current learning rate: 0.0039465366718084705
Test loss: 1.7488345881454816
Train mIoU: 0.11000606742996485
Test mIoU: 0.055031763311853214


Current learning rate: 0.0027398955659630434
Test loss: 1.7640762647599664
Train mIoU: 0.11555793680537339
Test mIoU: 0.06788594054777679


Current learning rate: 0.0014682736788600235
Test loss: 1.8223636147326792
Train mIoU: 0.12182981752714674
Test mIoU: 0.07081265606196334


[INFO] Total training time: 3099.795 seconds


In [ ]:
# latency and FPS

import time
from torch.utils.data import RandomSampler

iterations = 1000
random_sampler = RandomSampler(val_dataset, replacement=False, num_samples=iterations)
random_dataloader = DataLoader(val_dataset, batch_size=1, sampler=random_sampler)

latency = []
fps = []
with torch.no_grad():
    for inputs, _ in random_dataloader:
        inputs = inputs.cuda()
        start = time.time()
        prediction = bisenet(inputs)
        end = time.time()
        latency_i = end - start
        latency.append(latency_i)
        fps_i = 1/latency_i
        fps.append(fps_i)

meanLatency = np.mean(latency)*1000
stdLatency = np.std(latency)*1000
meanfps = np.mean(fps)
stdfps = np.std(fps)

print("Mean Latency:", meanLatency)
print("Std Latency:", stdLatency)
print("Mean FPS:", meanfps)
print("Std FPS:", stdfps)

In [ ]:
# FLOPs and parameters

from fvcore.nn import FlopCountAnalysis, flop_count_table

height = 512
width = 1024
image = torch.zeros((3, height, width))

flops = FlopCountAnalysis(bisenet, image)
print(flop_count_table(flops))

In [ ]:
def getting_val_miou(model, dataloader):

    model.eval()
    hist = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):

            inputs, targets = inputs.cuda(), targets.cuda()
            targets = torch.argmax(targets, dim=1)
            outputs = model(inputs)

            outputs = torch.argmax(outputs, dim=1)
            hist += total_hist(outputs, targets, 19)

    miou_per_class = per_class_iou(hist)
    miou = np.mean(miou_per_class)

    return miou, miou_per_class

In [ ]:
miou, miou_per_class = getting_val_miou(bisenet, val_dataloader)

print(f"Average mIoU:{miou}")
print(f"Average mIoU per class:{miou_per_class}")

Average mIoU:4.167839020662217e-05
Average mIoU per class:[0.00079189 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


##### Data Augmentation

In [ ]:
train_transform_aug1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([
        transforms.Compose([
            transforms.RandomRotation(degrees=30),
            transforms.RandomResizedCrop(size=(720, 1280), interpolation=transforms.InterpolationMode.NEAREST)
        ])
    ], p=0.5),
    transforms.Resize((720, 1280), interpolation=transforms.InterpolationMode.NEAREST),  # To make sure that the output is (720,1280) even if we do not apply augmentation
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform_aug2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.5),
    transforms.Resize((720, 1280), interpolation=Image.Resampling.NEAREST),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform_aug3 = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0))], p=0.5),
    transforms.Resize((720,1280), interpolation=Image.Resampling.NEAREST),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_lbl_transform = transforms.Compose([
    transforms.Resize((720,1280), interpolation=Image.Resampling.NEAREST),
])

In [ ]:
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((512,1024), interpolation=Image.Resampling.NEAREST),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_lbl_transform = transforms.Compose([
    transforms.Resize((512,1024), interpolation=Image.Resampling.NEAREST),
])

In [ ]:
train_dataset1 = GTA5('/content/drive/MyDrive/MLDL_project/datasets/GTA5', transform=[train_transform_aug1,train_lbl_transform]) # datasets\\GTA5
train_dataset2 = GTA5('/content/drive/MyDrive/MLDL_project/datasets/GTA5', transform=[train_transform_aug2,train_lbl_transform]) # datasets\\GTA5
train_dataset3 = GTA5('/content/drive/MyDrive/MLDL_project/datasets/GTA5', transform=[train_transform_aug3,train_lbl_transform]) # datasets\\GTA5
val_dataset = CityscapesDataset('/content/drive/MyDrive/MLDL_project/datasets/Cityspaces', transform=[val_transform,val_lbl_transform]) # datasets\\Cityscapes\\Cityspaces

train_dataloader1 = DataLoader(train_dataset1, batch_size=4, shuffle=True, num_workers = 8) #16
train_dataloader2 = DataLoader(train_dataset2, batch_size=4, shuffle=True, num_workers = 8) #16
train_dataloader3 = DataLoader(train_dataset3, batch_size=4, shuffle=True, num_workers = 8) #16
val_dataloader = DataLoader(val_dataset, split='val', batch_size=4, shuffle=True, num_workers = 8) #16

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
# Define loss and initial learning rate
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
initial_lr = 2.5e-2/4

aug1

In [ ]:
bisenet_aug1 = BiSeNet(num_classes = 19, context_path='resnet18').to(device)

# Define optimizer
optimizer_aug1 = torch.optim.SGD(bisenet_aug1.parameters(), lr=initial_lr, momentum=0.9, weight_decay=1e-4)

# Start the timer
start_time = timer()

# Setup training and save the results
epochs = 5 #50

miou_train_list_aug1 = []
miou_test_list_aug1 = []
miou_per_class_train_list_aug1 = []
miou_per_class_test_list_aug1 = []

for ep in range(epochs):

    train_loss, train_miou, train_miou_per_class = train(bisenet_aug1, optimizer_aug1, train_dataloader1, loss_fn)
    test_loss, test_miou, test_miou_per_class = test(bisenet_aug1, val_dataloader, loss_fn)

    ## Inserting learning rate decay
    curr_lr = poly_lr_scheduler(optimizer = optimizer_aug1, init_lr = initial_lr, iter = ep, lr_decay_iter=1, max_iter=epochs, power=0.9)

    miou_train_list_aug1.append(train_miou)
    miou_test_list_aug1.append(test_miou)
    miou_per_class_train_list_aug1.append(train_miou_per_class)
    miou_per_class_test_list_aug1.append(test_miou_per_class)

    print(f"Current learning rate: {curr_lr}")
    print(f"Test loss: {test_loss}")
    print(f"Train mIoU: {train_miou}")
    print(f"Test mIoU: {test_miou}")
    print("\n")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

aug2

In [ ]:
bisenet_aug2 = BiSeNet(num_classes = 19, context_path='resnet18').to(device)

# Define optimizer
optimizer_aug2 = torch.optim.SGD(bisenet_aug2.parameters(), lr=initial_lr, momentum=0.9, weight_decay=1e-4)

# Start the timer
start_time = timer()

# Setup training and save the results
epochs = 5 #50

miou_train_list_aug2 = []
miou_test_list_aug2 = []
miou_per_class_train_list_aug2 = []
miou_per_class_test_list_aug2 = []

for ep in range(epochs):

    train_loss, train_miou, train_miou_per_class = train(bisenet_aug2, optimizer_aug2, train_dataloader1, loss_fn)
    test_loss, test_miou, test_miou_per_class = test(bisenet_aug2, val_dataloader, loss_fn)

    ## Inserting learning rate decay
    curr_lr = poly_lr_scheduler(optimizer = optimizer_aug2, init_lr = initial_lr, iter = ep, lr_decay_iter=1, max_iter=epochs, power=0.9)

    miou_train_list_aug2.append(train_miou)
    miou_test_list_aug2.append(test_miou)
    miou_per_class_train_list_aug2.append(train_miou_per_class)
    miou_per_class_test_list_aug2.append(test_miou_per_class)

    print(f"Current learning rate: {curr_lr}")
    print(f"Test loss: {test_loss}")
    print(f"Train mIoU: {train_miou}")
    print(f"Test mIoU: {test_miou}")
    print("\n")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

aug3

In [ ]:
bisenet_aug3 = BiSeNet(num_classes = 19, context_path='resnet18').to(device)

# Define optimizer
optimizer_aug3 = torch.optim.SGD(bisenet_aug3.parameters(), lr=initial_lr, momentum=0.9, weight_decay=1e-4)

# Start the timer
start_time = timer()

# Setup training and save the results
epochs = 5 #50

miou_train_list_aug3 = []
miou_test_list_aug3 = []
miou_per_class_train_list_aug3 = []
miou_per_class_test_list_aug3 = []

for ep in range(epochs):

    train_loss, train_miou, train_miou_per_class = train(bisenet_aug3, optimizer_aug3, train_dataloader1, loss_fn)
    test_loss, test_miou, test_miou_per_class = test(bisenet_aug3, val_dataloader, loss_fn)

    ## Inserting learning rate decay
    curr_lr = poly_lr_scheduler(optimizer = optimizer_aug3, init_lr = initial_lr, iter = ep, lr_decay_iter=1, max_iter=epochs, power=0.9)

    miou_train_list_aug3.append(train_miou)
    miou_test_list_aug3.append(test_miou)
    miou_per_class_train_list_aug3.append(train_miou_per_class)
    miou_per_class_test_list_aug3.append(test_miou_per_class)

    print(f"Current learning rate: {curr_lr}")
    print(f"Test loss: {test_loss}")
    print(f"Train mIoU: {train_miou}")
    print(f"Test mIoU: {test_miou}")
    print("\n")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")